In [530]:
import pandas as pd
import glob
import unidecode as ud
import re

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

Abrimos el archivo del hashtag en pandas.

In [531]:
ht = pd.read_csv('felizmartes_clean.txt',sep='\t',on_bad_lines='skip')
print('Número de tweets:',len(ht))
ht.head(2)

Número de tweets: 12647


,id tweet,date,author,text,app,id user,followers,following,stauses,location,urls,geolocation,name,description,url_media,type media,quoted,relation,replied_id,user replied,retweeted_id,user retweeted,quoted_id,user quoted,first HT,lang,created_at,verified,avatar,link
0,1567551888861577216,2022-09-07 16:34:28,@rodrigp1964,"RT @Lonspe1: ¡¡ Éxito absoluto de Antonio Sánchez en su encuentro con la ciudadanía...!! 😂😂, Que te vote txapote #QueTeVoteTxapote 🤬 #FelizMartes Camboya Birmania https://t.co/oZ9OozYtIb",Twitter for Android,243744390,1113,112,105884,Las Palmas de Gran Canaria,None,None,Rodrigo Garcia,Nunca es tarde. 🇪🇸🇪🇸🇪🇸,http://pbs.twimg.com/ext_tw_video_thumb/1567043696314486786/pu/img/JmgUjeMB2yt7LInG.jpg,photo,None,RT,None,None,1567043749317951490,@Lonspe1,None,None,QueTeVoteTxapote,es,2011-01-27 18:30:39,False,https://pbs.twimg.com/profile_images/1397564222591078407/yHieeZCd_normal.jpg,https://twitter.com/rodrigp1964/status/1567551888861577216
1,1567551575471570944,2022-09-07 16:33:13,@anamferraez,#ResumenDeLaMañanera | 7 de septiembre #BuenosDías #FelizMartes A continuación les comparto los temas que fueron abordados esta mañana en la #ConferenciaMatutina. @lopezobrador_ https://t.co/uoWKfm04Qq https://t.co/5UCyjVtolX,Twitter for iPhone,61910503,3712,2114,15536,"Xalapa, Veracruz",https://m.facebook.com/story.php?story_fbid=pfbid0Y6izxrvoGeKWCDd4yyQeq8vzN4CAs2TCq1SMHWoJvN8j8LHC4gaAJ2Ln12fMa25pl&id=100044343880289,None,Ana Miriam Ferráez,"Diputada por el Distrito XI Xalapa II. Abogada, madre y emprendedora social.",http://pbs.twimg.com/media/FcEP2RuaQAAqzEH.jpg,photo,None,None,None,None,None,None,None,None,ResumenDeLaMañanera,es,2009-08-01 00:04:47,False,https://pbs.twimg.com/profile_images/1525205546122485761/FwI9ogiV_normal.jpg,https://twitter.com/anamferraez/status/1567551575471570944


Loopeamos para separar los hashtags del texto de los tweets. Con unidecode les quitamos las tildes.

In [532]:
ht['hashtags'] = ht['text'].apply(lambda x: re.findall(r"#(\w+)", x))
hashtag_list = ht['hashtags'].tolist()
ht_con_tildes = []
for listed in hashtag_list:
    for ht2 in listed:
        ht_con_tildes.append(ht2)
hashtag_list_separated = []
for ht2 in ht_con_tildes:
    ht2 = ud.unidecode(ht2)
    hashtag_list_separated.append(ht2)

print(hashtag_list_separated[:5], '...')

['QueTeVoteTxapote', 'FelizMartes', 'ResumenDeLaMananera', 'BuenosDias', 'FelizMartes'] ...


Lo convertimos a dataframe y lo pasamos todo a minúsculas.

In [533]:
ht_list_df = pd.DataFrame (hashtag_list_separated, columns = ['Label']) 
ht_list_df['Label'] = ht_list_df['Label'].str.lower()
ht_list_df

,Label
0,quetevotetxapote
1,felizmartes
2,resumendelamananera
3,buenosdias
4,felizmartes
...,...
41087,martianmtmad14
41088,violacomeilmarevenezia79
41089,buenosdias
41090,felizmartes


Contamos la cantidad de veces que aparece cada hashtag y reseteamos el index para introducirlo como columna.

In [534]:
nodes_gdf = ht_list_df.value_counts().to_frame().reset_index().reset_index()
nodes_gdf

,index,Label,0
0,0,felizmartes,12662
1,1,6sep,1522
2,2,somospatriaqueserespeta,1202
3,3,buenosdias,703
4,4,quetevotetxapote,674
...,...,...,...
3807,3807,compania,1
3808,3808,miinfanciaenelrecuerdo,1
3809,3809,migracioncontigo,1
3810,3810,migracion,1


Si hay muchos podemos filtrar para quedarnos con los que aparezcan más de `number` veces.

In [535]:
filter_question = input('¿Filtrar hashtags por número de apariciones? y/n: ')
if filter_question == 'y':
    number = input('Mantener hashtags con más de __ apariciones. Inserte número: ')
    nodes_gdf = nodes_gdf[nodes_gdf[0] >= int(number)]
    filtered_list = nodes_gdf['Label'].tolist()

Cambiamos los nombres de las columnas a 0, 1 y 2 porque luego concatenaremos el dataframe de ejes con otras columnas. Aunque parezca extraño esto es necesario para que Gephi reconozca las partes del archivo gdf. Esos valores marcarán el inicio de la parte de nodos. Concatenamos `nodes_gdf` a `nodes_gdf_first_row_df`.

In [536]:
nodes_gdf.columns = [0,1,2]
nodes_gdf_first_row = {0: ['nodedef>name VARCHAR'],
        1: ['label VARCHAR'],
        2: ['Links INT']}
nodes_gdf_first_row_df = pd.DataFrame(nodes_gdf_first_row)
nodes_gdf = pd.concat([nodes_gdf_first_row_df,nodes_gdf],axis=0)
nodes_gdf

,0,1,2
0,nodedef>name VARCHAR,label VARCHAR,Links INT
0,0,felizmartes,12662
1,1,6sep,1522
2,2,somospatriaqueserespeta,1202
3,3,buenosdias,703
...,...,...,...
3807,3807,compania,1
3808,3808,miinfanciaenelrecuerdo,1
3809,3809,migracioncontigo,1
3810,3810,migracion,1


Construimos una función para que a partir de un dataframe construya relaciones entre los hashtags que aparecen en un tweet. El primer hashtag usado será la "Source" y los demás serán los "Targets". Si por ejemplo un tweet tiene 3 hashtags como #felizmartes #buenosdias #alegria se crearán dos relaciones: #felizmartes -> #buenosdias y #felizmartes -> #alegria.

In [537]:
def build_edges(row):
    source = row['first HT']
    Labels = row['hashtags']
    for Label in Labels:
        edges_list.append([source,Label])


Aplicamos la función a nuestro hashtag y obtenemos las relaciones.

In [538]:
edges_list = []
ht.apply(build_edges, axis=1)
edges_df = pd.DataFrame (edges_list, columns = ['Source','Label'])
edges_df


,Source,Label
0,QueTeVoteTxapote,QueTeVoteTxapote
1,QueTeVoteTxapote,FelizMartes
2,ResumenDeLaMañanera,ResumenDeLaMañanera
3,ResumenDeLaMañanera,BuenosDías
4,ResumenDeLaMañanera,FelizMartes
...,...,...
41087,APOYOROCIO6S,MartianMTMAD14
41088,APOYOROCIO6S,ViolaComeIlMareVenezia79
41089,Buenosdias,Buenosdias
41090,Buenosdias,Felizmartes


Pasamos las dos columnas a minúsculas y les quitamos las tildes.

In [539]:
edges_df['Source'] = edges_df['Source'].apply(lambda x:ud.unidecode(x))
edges_df['Source'] = edges_df['Source'].str.lower()
edges_df['Label'] = edges_df['Label'].apply(lambda x:ud.unidecode(x))
edges_df['Label'] = edges_df['Label'].str.lower()

edges_df


,Source,Label
0,quetevotetxapote,quetevotetxapote
1,quetevotetxapote,felizmartes
2,resumendelamananera,resumendelamananera
3,resumendelamananera,buenosdias
4,resumendelamananera,felizmartes
...,...,...
41087,apoyorocio6s,martianmtmad14
41088,apoyorocio6s,violacomeilmarevenezia79
41089,buenosdias,buenosdias
41090,buenosdias,felizmartes


Quitamos las relaciones donde el hashtag "Source" es el mismo que el "Target".

In [540]:
edges_df = edges_df[edges_df['Source'] != edges_df['Label']]
edges_df

,Source,Label
1,quetevotetxapote,felizmartes
3,resumendelamananera,buenosdias
4,resumendelamananera,felizmartes
5,resumendelamananera,conferenciamatutina
7,vivamexico,felizmartes
...,...,...
41086,apoyorocio6s,krashxerik
41087,apoyorocio6s,martianmtmad14
41088,apoyorocio6s,violacomeilmarevenezia79
41090,buenosdias,felizmartes


Si hemos filtrado los hashtags por número tenemos que filtrarlos también en el dataframe de los ejes.

In [541]:
if filter_question == 'y':
    edges_df = edges_df[edges_df['Source'].isin(filtered_list)]
    edges_df = edges_df[edges_df['Label'].isin(filtered_list)]

ids = ht_list_df.value_counts().to_frame().reset_index().reset_index()
ids = ids[['Label','index']]

Generamos un dataframe que tenga solo el index de cada hashtag para poder identificarlo. Por comodidad hemos dejado la columna 'Target' como 'Label' para poder hacer el merge directamente. Con este merge le asociamos la identidad numérica a cada hashtag.

In [542]:
ids = ht_list_df.value_counts().to_frame().reset_index().reset_index()
ids = ids[['Label','index']]
ids

,Label,index
0,felizmartes,0
1,6sep,1
2,somospatriaqueserespeta,2
3,buenosdias,3
4,quetevotetxapote,4
...,...,...
3807,compania,3807
3808,miinfanciaenelrecuerdo,3808
3809,migracioncontigo,3809
3810,migracion,3810


Le asociamos el index a la columna 'Label', que luego será 'Target'

In [543]:
edges_df = edges_df.merge(ids,how='left',on='Label')
edges_df

,Source,Label,index
0,quetevotetxapote,felizmartes,0
1,resumendelamananera,buenosdias,3
2,resumendelamananera,felizmartes,0
3,resumendelamananera,conferenciamatutina,3609
4,vivamexico,felizmartes,0
...,...,...,...
28384,apoyorocio6s,krashxerik,566
28385,apoyorocio6s,martianmtmad14,572
28386,apoyorocio6s,violacomeilmarevenezia79,742
28387,buenosdias,felizmartes,0


Quitamos la columna 'Label' y renombramos la columna 'Index' como 'Target', pues lo que nos interesa es su código numérico.

In [544]:
edges_df = edges_df.rename(columns={'index':'Target'})
edges_df = edges_df.drop('Label',axis=1)
edges_df

,Source,Target
0,quetevotetxapote,0
1,resumendelamananera,3
2,resumendelamananera,0
3,resumendelamananera,3609
4,vivamexico,0
...,...,...
28384,apoyorocio6s,566
28385,apoyorocio6s,572
28386,apoyorocio6s,742
28387,buenosdias,0


Ahora al dataframe `ids` le cambiamos el nombre de la columna 'Label' a 'Source' para asociar el código numérico de nuevo.

In [545]:
ids.columns = ['Source','index']
ids

,Source,index
0,felizmartes,0
1,6sep,1
2,somospatriaqueserespeta,2
3,buenosdias,3
4,quetevotetxapote,4
...,...,...
3807,compania,3807
3808,miinfanciaenelrecuerdo,3808
3809,migracioncontigo,3809
3810,migracion,3810


Lo juntamos con nuestro dataframe `edges_df`

In [546]:
edges_df = edges_df.merge(ids,how='left',on='Source')

Borramos la columna 'Source' y a la 'index' le ponemos el nombre de 'Source', pues lo que nos interesa es el código numérico.

In [547]:
edges_df = edges_df.drop('Source',axis=1)
edges_df = edges_df.rename(columns={'index':'Source'})
edges_df

,Target,Source
0,0,4
1,3,2164
2,0,2164
3,3609,2164
4,0,9
...,...,...
28384,566,170
28385,572,170
28386,742,170
28387,0,3


Reordenamos las columnas y añadimos la columna 'Type' con el valor false. Esto le dirá a Gephi que no hay una relación directa pues, en mi experiencia, que un hashtag aparezca el primero no implica que sea el principal.

In [548]:
edges_df = edges_df[['Source','Target']]
edges_df['Type'] = 'false'
edges_df

,Source,Target,Type
0,4,0,false
1,2164,3,false
2,2164,0,false
3,2164,3609,false
4,9,0,false
...,...,...,...
28384,170,566,false
28385,170,572,false
28386,170,742,false
28387,3,0,false


Creamos otro dataframe con esos valores porque los necesitamos para que señalen a Gephi donde empieza la parte de ejes. Es parte de la estructura del archivo gdf. Concatenamos nuestro dataframe de ejes a ese.

In [549]:
edges_gdf = edges_df
edges_gdf.columns = [0,1,2]

data = {0: ['edgedef>node1 VARCHAR'],
        1: ['node2 VARCHAR'],
        2: ['directed BOOLEAN']}
columns = pd.DataFrame(data)
edges_gdf = pd.concat([columns,edges_df],axis=0)
edges_gdf

,0,1,2
0,edgedef>node1 VARCHAR,node2 VARCHAR,directed BOOLEAN
0,4,0,false
1,2164,3,false
2,2164,0,false
3,2164,3609,false
...,...,...,...
28384,170,566,false
28385,170,572,false
28386,170,742,false
28387,3,0,false


Por último concatenamos el dataframe de nodos y ejes para montar nuestro archivo gdf.

In [550]:
gdf = pd.concat([nodes_gdf,edges_gdf],axis=0)
gdf

,0,1,2
0,nodedef>name VARCHAR,label VARCHAR,Links INT
0,0,felizmartes,12662
1,1,6sep,1522
2,2,somospatriaqueserespeta,1202
3,3,buenosdias,703
...,...,...,...
28384,170,566,false
28385,170,572,false
28386,170,742,false
28387,3,0,false


Y lo guardamos como gdf. Gephi puede abrir este archivo directamente y entender cuáles son los nodos y cuáles los ejes. Debemos hacerlo así y no con un archivo csv de nodos y otro de ejes porque aunque podamos representar los grafos no podremos modificar los datos para añadir por ejemplo la modularidad, que es clave para nuestro análisis. Ejemplo: en mi hashtag tengo 4 nubes de usuarios y me interesa quedarme solo con 2 para tener resultados más limpios. En Gephi aplico la modularidad y exporto la tabla de nodos. En esa tabla tendré el hashtag o el usuario (depende del grafo) y luego la uso para añadir la modularidad a mi dataframe del hashtag y ya puedo filtrar mis 2 nubes de interés por modularidad y hacer el análisis con Python.

In [551]:
gdf.to_csv('grafo.gdf',sep=',',index=False,header=False)